# Install datasets
Restart runtime after installing libraries.

In [1]:
!pip install datasets
!pip install transformers
!pip install accelerate

# Download dataset and model
Download dataset from huggingface or get local dataset

In [2]:
# huggingface dataset
from datasets import load_dataset
dataset = load_dataset("tatsu-lab/alpaca")

In [ ]:
dataset

,Unnamed: 0,question,answer,source
0,0,Đưa ra ba lời khuyên để duy trì sức khoẻ.,"1. Áp dụng chế độ ăn cân bằng, đảm bảo ăn nhi...",alpaca vi
1,1,Ba màu cơ bản là gì?,"Ba màu cơ bản là đỏ, xanh dương và vàng.",alpaca vi
2,2,Mô tả cấu trúc của một nguyên tử.,"Một nguyên tử được tạo thành từ một hạt nhân,...",alpaca vi
3,3,Làm thế nào để giảm ô nhiễm không khí?,"Có một số cách để giảm ô nhiễm không khí, chẳ...",alpaca vi
4,4,Giả vờ là giám đốc dự án của một công ty xây ...,Khi còn là giám đốc dự án tại một công ty xây...,alpaca vi
...,...,...,...,...
30702,30702,Ai Là Người Phát Minh Ra Biến Áp Điện?,Biến Áp Điện Được Phát Minh Bởi Người Anh Tên ...,chunk_26053
30703,30703,Người Nào Đã Tìm Ra Dụng Cụ Bằng Đá Đầu Tiên?,Người Neanderthal Đã Tìm Ra Dụng Cụ Bằng Đá Đầ...,chunk_21813
30704,30704,Ai Là Người Đã Viết Bài Báo Được Trích Dẫn?,Người Viết Bài Báo Được Trích Dẫn Là Mark Smith.,chunk_22450
30705,30705,Người Nào Đã Viết Cuốn Sách 'Tiếng Chim Hót Tr...,Người Viết Cuốn Sách 'Tiếng Chim Hót Trong Bụi...,chunk_37786


# Download model

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

torch.manual_seed(42)
output_dir = "EleutherAI/gpt-neo-125m"
tokenizer = AutoTokenizer.from_pretrained(output_dir, bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = AutoModelForCausalLM.from_pretrained(output_dir, low_cpu_mem_usage=True, device_map="auto") # .cuda() torch_dtype=torch.float16,
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

Embedding(50259, 768)

# Preprocess

In [6]:
dataset = dataset['train']

KeyError: ignored

In [19]:
data_list = []
text = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
for i in range(50): # len(dataset['instruction'])):
    if dataset['input']:
        temp = text + "###Instruction:" + dataset['instruction'][i] + "\n\n###Input:" + dataset['input'][i] + "\n\n###Output:" + dataset['output'][i]
    else:
        temp = text + "###Instruction:" + dataset['instruction'][i] + "\n\n###Output:" + dataset['output'][i]
    data_list.append(temp)

In [20]:
data_list = pd.Series(data_list)

In [21]:
data_list[0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.###Instruction:Give three tips for staying healthy.\n\n###Input:\n\n###Output:1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'

In [22]:
len(data_list)

50

In [23]:
max_length = max([len(tokenizer.encode(e)) for e in data_list])
print("Max length: {}".format(max_length))

Max length: 349


In [24]:
class MyDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer(f"<|startoftext|>"+ txt + "<|endoftext|>", truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [25]:
mydataset = MyDataset(data_list, tokenizer, max_length = max_length)

In [26]:
mydataset

# Train

In [27]:
import gc
import torch
from transformers import DataCollatorForLanguageModeling, AutoConfig, default_data_collator
gc.collect()
torch.cuda.empty_cache()

In [ ]:
training_args = TrainingArguments(
    output_dir = "finetune_save",
    optim="adafactor", # 메모리 사용 감소
    num_train_epochs=1,
    per_device_train_batch_size=2,
    # fp16=True,
    learning_rate=5e-5,
    logging_strategy="steps",
    warmup_steps=3000,
    weight_decay=0.01,
    logging_steps=0.1,
    save_steps=0.06, # 소수점은 퍼센트, 정수면 스텝수
    save_strategy="steps",
    report_to="none"
)

trainer = Trainer(model=model, args=training_args, train_dataset=mydataset,
    data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])})

trainer.train()

Step,Training Loss
3,13.546400
6,21.326500
9,20.882600
12,19.904700


# Save model

In [ ]:
output_dir = "./FT/diff_tokenizer"

In [ ]:
# save as trainer
# tokenizer.save_pretrained(output_dir)
# trainer.save_model(output_dir)

In [ ]:
# save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./FT/diff_tokenizer\\tokenizer_config.json',
 './FT/diff_tokenizer\\special_tokens_map.json',
 './FT/diff_tokenizer\\tokenizer.json')

# Load saved model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# tokenizer = AutoTokenizer.from_pretrained(output_dir)
# model = AutoModelForCausalLM.from_pretrained(output_dir, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device=f"cuda", non_blocking=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained(output_dir)
# tokenizer = AutoTokenizer.from_pretrained("eunyounglee/GPT-NeoX-1.3B-Viet-finetune-epoch-10")
model = AutoModelForCausalLM.from_pretrained(output_dir).to("cuda")

# Load Dataset again

In [ ]:
# from datasets import load_dataset
# dataset = load_dataset("tatsu-lab/alpaca")
# dataset = dataset['train']

# Generate 1

In [ ]:
i = 0
text = "###Question:" + dataset['question'][i] + "\n###Answer:"

In [ ]:
text

'###Question: Đưa ra ba lời khuyên để duy trì sức khoẻ.\n###Answer:'

In [ ]:
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "left"
generated = tokenizer("<|startoftext|>" + text, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                            #     bos_token='<|startoftext|>',
                             #    eos_token='<|endoftext|>', pad_token='[PAD]', padding_side="left",
                                max_length=300, top_p=0.95, temperature=1.0, num_return_sequences=5)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {} \n".format(i, tokenizer.decode(
        sample_output, skip_special_tokens=True)))

0:  ###Question: Đưa ra ba lời khuyên để duy trì sức khoẻ.
###Answer: để và làm để việc công. công lượng mà bạn số có như là sử một việc để ở các liên lượng
 sự và như làm đó đến từ có khác trong từ một từ. không làm thực các như để là để người việc
 của trong cách. sự sử sự. nó
 và có, việc giá để 

1:  ###Question: Đưa ra ba lời khuyên để duy trì sức khoẻ.
###Answer:,. không. công,, đến các mà ở. để và., thực những phát hiện của phát từ: làm nhiều không lượng trong cho sự 

2:  ###Question: Đưa ra ba lời khuyên để duy trì sức khoẻ.
###Answer: có là là trong công lượng cho không làm để lượng cơ để một một
 thực cho thực thức có sự: để sử sự các phát là sự một thực trong thành thực có thực sự người ở không làm. nhà
 sự thực
 trong, công học có thức lý học thị. không và phát
 thực và các để ở nhiều của số và học thành cái học là sản thức các học thức mà phát thành sinh học sự lớn những thức là công sự những thực sự của của liên có quan thức của cái của của quan trong giá lớn phát làm cá

# Generate 2

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0) # device=0)

In [ ]:
prompt3 = text #@param {type:"string"}
response_min_chars =  20#@param {type:"integer"}
response_max_chars =  200#@param {type:"integer"}

response_3 = generator(prompt3, do_sample=True, min_length=response_min_chars,
                       max_length=response_max_chars,
                       clean_up_tokenization_spaces=True,
                       return_full_text=True)
out3_dict = response_3[0]
out3_dict

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


{'generated_text': '###Question: Bạn có thể tìm thấy gì trên Wikipedia?\n###Answer:  có�ca đ+ địột:ột đị 1ột h�á đ+ h� P+ đị sở+� đ Nhung h�) có đ Nhung sạch đ+ Văn bộầột có) bộỹ đ) sở ph+ đị� đ Nhung X)õ Nhung Thương đ) bộếôi+� đ Nhung X)õ�ôi� 1 đ có)õ sạch) đ có) đ�ôi luôn� đ Nhung cùng giaoõ Nhung hướng đ) P Nhung sạch� đ Nhung X)�i Nhungờôi)õ Nhung cùng P)�ột h�)õ h�)õ đị) P)� nặng)� sở ph)ổ P�ôi)�ược�ờ P)�ôi7�� đ Nhung X)õ7 quốc đ)�ôi7�� đ Nhung X'}

# Upload to Huggingface

In [ ]:
from huggingface_hub import login
login()

In [ ]:
tokenizer.push_to_hub("eunyounglee/GPT-NeoX-1.3B-viet-custom-10")

CommitInfo(commit_url='https://huggingface.co/eunyounglee/GPT-NeoX-1.3B-viet-custom-10/commit/c582c97f291cf6d43d6d1156c9ae7235c9279473', commit_message='Upload tokenizer', commit_description='', oid='c582c97f291cf6d43d6d1156c9ae7235c9279473', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub("eunyounglee/GPT-NeoX-1.3B-viet-custom-10")

pytorch_model.bin:   0%|          | 0.00/5.34G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/eunyounglee/GPT-NeoX-1.3B-viet-custom-10/commit/5984fbb02756288a08b1cc0b3895d2a262ffb26c', commit_message='Upload GPTNeoForCausalLM', commit_description='', oid='5984fbb02756288a08b1cc0b3895d2a262ffb26c', pr_url=None, pr_revision=None, pr_num=None)